In [1]:
from datasets import load_from_disk
import os

In [2]:
dataset_path = "/kaggle/input/convulation-dataset/conv_data"

In [3]:
# Load datasets
train_dataset = load_from_disk(os.path.join(dataset_path, "therapy_train"))
val_dataset = load_from_disk(os.path.join(dataset_path, "therapy_val"))
test_dataset = load_from_disk(os.path.join(dataset_path, "therapy_test"))

In [4]:
# Function to print 5 examples from a dataset
def print_examples(dataset, name):
    print(f"\n{name} Dataset Samples:")
    for i in range(5):
        print(dataset[i])  # Prints the first 5 examples

# Print examples
print_examples(train_dataset, "Train")
print_examples(val_dataset, "Validation")
print_examples(test_dataset, "Test")



Train Dataset Samples:
{'input_text': 'T: Hi you how to do it today? [SEP] P: Great. How are you?', 'target_text': "I'm doing well. Thanks for asking."}
{'input_text': "T: Hi you how to do it today? [SEP] P: Great. How are you? [SEP] T: I'm doing well. Thanks for asking.", 'target_text': "So you're doing great."}
{'input_text': "T: Hi you how to do it today? [SEP] P: Great. How are you? [SEP] T: I'm doing well. Thanks for asking. [SEP] T: So you're doing great. [SEP] P: I'm doing awesome.", 'target_text': "I know your brother brought you in today and he had expressed some concerns about your mood. Do you know what that's about?"}
{'input_text': "T: Hi you how to do it today? [SEP] P: Great. How are you? [SEP] T: I'm doing well. Thanks for asking. [SEP] T: So you're doing great. [SEP] P: I'm doing awesome. [SEP] T: I know your brother brought you in today and he had expressed some concerns about your mood. Do you know what that's about? [SEP] P: I, I think he's worrying for no reason. 

In [5]:
!pip install transformers datasets bleu bert-score torch


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.2 MB/s eta 0:00:00
  Created wheel for bleu: filename=bleu-0.3-py3-none-any.whl size=5781 sha256=1cd076bbde8f9b90c026607576dc090be2e54c48d2692e1140e0cadf190889e0
  Stored in directory: /root/.cache/pip/wheels/c6/d8/d1/009abe01b8b2c6a14c62d197b510b3cc1076014c22d712c5ce
Successfully built bleu


In [6]:
import torch
from datasets import load_from_disk
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import Dataset
import os



In [7]:
# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Dataset class for tokenizing input & target text
class ConversationDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        input_text = "predict next utterance: " + sample["input_text"]
        target_text = sample["target_text"]

        # Tokenize input and target
        inputs = self.tokenizer(
            input_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        targets = self.tokenizer(
            target_text, max_length=50, padding="max_length", truncation=True, return_tensors="pt"
        )

        return {
            "input_ids": inputs["input_ids"].squeeze(),
            "attention_mask": inputs["attention_mask"].squeeze(),
            "labels": targets["input_ids"].squeeze()
        }

# Convert datasets to PyTorch format
train_data = ConversationDataset(train_dataset, tokenizer)
val_data = ConversationDataset(val_dataset, tokenizer)
test_data = ConversationDataset(test_dataset, tokenizer)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
# Load pre-trained model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Define training parameters
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=2,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to=[],

)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer
)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-8-f0f228473480>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
trainer.train()


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,1.691600,1.650402
2,1.654100,1.618708
3,1.652500,1.607143
4,1.648200,1.599849
5,1.648100,1.594764
6,1.625100,1.591039
7,1.626700,1.588509
8,1.561700,1.587165
9,1.626300,1.586113
10,1.595800,1.585770


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=5010, training_loss=1.714911717854574, metrics={'train_runtime': 1262.3768, 'train_samples_per_second': 31.75, 'train_steps_per_second': 3.969, 'total_flos': 5424499402997760.0, 'train_loss': 1.714911717854574, 'epoch': 10.0})

In [10]:
model.save_pretrained("/kaggle/working/t5_therapy_model")
tokenizer.save_pretrained("/kaggle/working/t5_therapy_model")


('/kaggle/working/t5_therapy_model/tokenizer_config.json',
 '/kaggle/working/t5_therapy_model/special_tokens_map.json',
 '/kaggle/working/t5_therapy_model/spiece.model',
 '/kaggle/working/t5_therapy_model/added_tokens.json')

In [11]:
def generate_response(model, tokenizer, text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    input_text = "predict next utterance: " + text
    input_ids = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True).input_ids.to(device)
    
    output_ids = model.generate(input_ids, max_length=50, num_beams=5, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)


# Test predictions
for i in range(5):
    sample_input = test_dataset[i]["input_text"]
    actual_output = test_dataset[i]["target_text"]
    generated_output = generate_response(model, tokenizer, sample_input)

    print(f"\nInput: {sample_input}")
    print(f"Actual: {actual_output}")
    print(f"Generated: {generated_output}")



Input: T: So, alright, let's take a step back and talk about competency. What education Did you need for the job you have now? [SEP] P: I have my bachelor's degree. And we got a little bit of training. You know every so often we get some training at work and stuff like that.
Actual: Are you evaluated at work by anybody to see if you're in a job you should be?
Generated: Okay, let's take a step back and talk about competency.

Input: T: So, alright, let's take a step back and talk about competency. What education Did you need for the job you have now? [SEP] P: I have my bachelor's degree. And we got a little bit of training. You know every so often we get some training at work and stuff like that. [SEP] T: Are you evaluated at work by anybody to see if you're in a job you should be? [SEP] P: Yeah, I have a supervisor so they check up on stuff and also like if I feel like I have questions and stuff like that, I can go to them as well.
Actual: Have you been Found in that system to be som

In [12]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:0000:01


In [13]:
import evaluate
from bert_score import score
from tqdm import tqdm

# Load metrics
bleu = evaluate.load("bleu")

# Optional: also load BERTScore from Hugging Face if you prefer
# bertscore = evaluate.load("bertscore")

# Helper function to generate model predictions
def generate_response(model, tokenizer, input_text, max_length=32):
    model.eval()
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Prepare predictions and references
references = []
predictions = []

for i in tqdm(range(len(test_dataset))):
    input_text = test_dataset[i]["input_text"]
    target_text = test_dataset[i]["target_text"]

    prediction = generate_response(model, tokenizer, input_text)
    
    references.append(target_text)
    predictions.append(prediction)

# BLEU Score
bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
print("\nBLEU Score:", bleu_score["bleu"])

# BERTScore
P, R, F1 = score(predictions, references, lang="en", verbose=True)
print("\nBERTScore:")
print("F1 mean:", F1.mean().item())
print("Precision mean:", P.mean().item())
print("Recall mean:", R.mean().item())


100%|██████████| 968/968 [02:30<00:00,  6.43it/s]



BLEU Score: 0.0020127217198788933


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/19 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/16 [00:00<?, ?it/s]

done in 5.41 seconds, 178.81 sentences/sec

BERTScore:
F1 mean: 0.8521084189414978
Precision mean: 0.8659338355064392
Recall mean: 0.8393159508705139
